# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import os
import math as mt
from astropy.modeling import models
from astropy import units as u
from scipy.optimize import curve_fit
from astropy.timeseries import LombScargle
from scipy.special import voigt_profile
import Analysis_functions as AF
from importlib import reload
import scipy.stats as ss
import matplotlib.colors as mc
import csv
import pandas as pd
from scipy.interpolate import interp1d
import bindensity as bind
import itertools
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
import spectrum_model as spec_mod

# Some preliminary info.

In [ ]:
day = '2023-01-26'
new = True
location = '/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/new_Solar-data/'+day
os.chdir(location)
if not os.path.exists('Plot_Results'):
    os.mkdir('Plot_Results')
    
#Defining directories
blaze_dir = 'Blaze-data'
telluric_dir = 'Telluric-data'
dir_name = 'Corr-data'
CCF_dir_name = 'CCF_RV-data'
rassine_dir_name = 'Rassine-data'
Rassine = True

#Continuum Ranges used for normalization and EW calculation
wav_ranges = [(10804.8, 10805.9), (10807.4, 10808.5), (10825.5, 10826.5), (10839, 10840), (10842.4, 10843.1), (10850.2, 10851.2), (10856.5, 10857.9)]

## Test 1

In [ ]:
test = pf.open(dir_name+'/'+sorted(os.listdir(dir_name))[10])

test.info()

In [ ]:
test[4].data[14]

## Test 2

In [ ]:
test_RV = pf.open(telluric_dir+'/'+sorted(os.listdir(telluric_dir))[10])

test_RV.info()

In [ ]:
test_RV[2].data[14]

In [ ]:
plt.plot(test_RV[2].data[14], test_RV[6].data[14])
plt.xlim(10830, 10840)

## Extracting data

In [ ]:
#Re-make this cell to make it smaller
reload(AF)

#Extracting the data
if new:
    print('New Extraction')
    if Rassine:
        print('Rassine')
        if os.path.exists(location+'/'+rassine_dir_name+'-HA'):
            print('Multiple modes')
            tot_lamda, rassine_total_lamda_HA, rassine_total_lamda_HE, rassine_master_lamda_HA, rassine_master_lamda_HE, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr_HA, rassine_total_norm_spctr_HE, rassine_master_spctr_HA, rassine_master_spctr_HE, tot_err, tot_norm_err, rassine_total_norm_err_HA, rassine_total_norm_err_HE, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, Rassine)
        else:
            print('Single mode')
            tot_lamda, rassine_total_lamda, rassine_master_lamda, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr, rassine_master_spctr, tot_err, tot_norm_err, rassine_total_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, Rassine)
    else:
        print('No Rassine')
        tot_lamda, tot_spctr, tot_norm_spctr, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, Rassine)
else:
    print('Old Extraction')
    tot_lamda, tot_norm_spctr, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM = AF.extraction(dir_name, blaze_dir, CCF_dir_name, 14)


# Reducing the data

## SNR and RV clipping

In [ ]:
reload(AF)
sig = 200
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
    total_SNR_HA, total_SNR_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)
    
    #Special ones - BEGIN
    tota_norm_spctr_HA, tota_norm_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

    tota_lamda_HA, tota_lamda_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)
    
    tota_norm_err_HA, tota_norm_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)
    
    tota_spctr_HA, tota_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)
        
    tota_err_HA, tota_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)
    
    tota_telluric_HA, tota_telluric_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
    #Special ones - END

    date_HA, date_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

    total_RV_HA, total_RV_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

    total_RV_err_HA, total_RV_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

    total_FWHM_HA, total_FWHM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

    total_FWHM_err_HA, total_FWHM_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_HA, total_BIS_SPAN_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_err_HA, total_BIS_SPAN_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

    total_CONTRAST_HA, total_CONTRAST_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

    total_CONTRAST_err_HA, total_CONTRAST_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

    total_H2O_HA, total_H2O_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

    total_H2O_err_HA, total_H2O_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

    total_O2_HA, total_O2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)
    
    total_O2_err_HA, total_O2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

    total_CO2_HA, total_CO2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

    total_CO2_err_HA, total_CO2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

    total_AIRM_HA, total_AIRM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)
    
#If there is one mode of observation
else:
    #We don't need to segment and reduce the RASSINE spectra because they were made from 
    #spectra that were previously segmented and reduced. 
    
    #Special ones - BEGIN
    tota_norm_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

    tota_lamda = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)
    
    tota_norm_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)
    
    tota_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)
        
    tota_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)
    
    tota_telluric = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
    #Special ones - END
    
    total_SNR = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)

    date = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

    total_RV = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

    total_RV_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

    total_FWHM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

    total_FWHM_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

    total_CONTRAST = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

    total_CONTRAST_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

    total_H2O = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

    total_H2O_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

    total_O2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)
    
    total_O2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

    total_CO2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

    total_CO2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

    total_AIRM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)
    
mode = tot_mode

## Interpolating everything on a common wavelength grid

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):

    min_lamda = max(max(tota_lamda_HA[:, 0]), max(tota_lamda_HE[:, 0]))
    max_lamda = min(min(tota_lamda_HA[:, -1]), min(tota_lamda_HE[:, -1]))
    
    ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda_HA[0]))
    
    total_lamda_HA = np.zeros(tota_lamda_HA.shape)
    total_lamda_HE = np.zeros(tota_lamda_HE.shape)
    
    total_norm_spctr_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_norm_spctr_HE = np.zeros(tota_norm_spctr_HE.shape)
    
    total_norm_err_HA = np.zeros(tota_norm_err_HA.shape)
    total_norm_err_HE = np.zeros(tota_norm_err_HE.shape)
    
    total_spctr_HA = np.zeros(tota_spctr_HA.shape)
    total_spctr_HE = np.zeros(tota_spctr_HE.shape)

    total_err_HA = np.zeros(tota_err_HA.shape)
    total_err_HE = np.zeros(tota_err_HE.shape)

    #ANTARESS
    #Re-making the wavelength array so that it is the correct format for Antaress
    preliminary_lamda_HA=np.zeros((tota_lamda_HA.shape[0], tota_lamda_HA.shape[1]+1))

    preliminary_lamda_HE=np.zeros((tota_lamda_HE.shape[0], tota_lamda_HE.shape[1]+1))

    for i in range(len(preliminary_lamda_HA)):
        preliminary_lamda_HA[i] = np.append(tota_lamda_HA[i], tota_lamda_HA[i][-1]+np.diff(tota_lamda_HA[i])[-1])

    for i in range(len(preliminary_lamda_HE)):
        preliminary_lamda_HE[i] = np.append(tota_lamda_HE[i], tota_lamda_HE[i][-1]+np.diff(tota_lamda_HE[i])[-1])

    #Re-making the reference wavelength vector for Antaress
    ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

    #Initializing variables
    #Normalized
    total_norm_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_norm_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

    total_norm_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
    total_norm_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)

    #Un-normalized (for SNR calculation)
    total_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

    total_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
    total_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)
    
    total_telluric_HA = np.zeros(tota_telluric_HA.shape)
    total_telluric_HE = np.zeros(tota_telluric_HE.shape)

    #HA
    for i in range(len(tota_norm_spctr_HA)):
        
        interpol_s = interp1d(tota_lamda_HA[i], tota_norm_spctr_HA[i])
        total_norm_spctr_HA[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda_HA[i], tota_norm_err_HA[i]**2)
        total_norm_err_HA[i] = np.sqrt(interpol_err(ref_wave))
            
        interpol_spctr = interp1d(tota_lamda_HA[i], tota_spctr_HA[i])
        total_spctr_HA[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda_HA[i], tota_err_HA[i]**2)
        total_err_HA[i] = np.sqrt(interpol_error(ref_wave))

        total_lamda_HA[i] = ref_wave
        
        #Populating ANTARESS arrays
        #Normalized
        testA = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_norm_spctr_HA[i], cov = np.array([tota_norm_err_HA[i]**2]) , kind='cubic')

        total_norm_spctr_Antaress_HA[i] = testA[0]

        total_norm_err_Antaress_HA[i] = np.sqrt(testA[1][0])
        
        #Un-normalized
        testB = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_spctr_HA[i], cov = np.array([tota_err_HA[i]**2]) , kind='cubic')
        
        total_spctr_Antaress_HA[i] = testB[0]
        
        total_err_Antaress_HA[i] = np.sqrt(testB[1][0])
        
        testC = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_telluric_HA[i], kind='cubic')
        
        total_telluric_HA[i] = testC
       
    #HE
    for i in range(len(tota_norm_spctr_HE)):
        
        interpol_s = interp1d(tota_lamda_HE[i], tota_norm_spctr_HE[i])
        total_norm_spctr_HE[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda_HE[i], tota_norm_err_HE[i]**2)
        total_norm_err_HE[i] = np.sqrt(interpol_err(ref_wave))
        
        interpol_spctr = interp1d(tota_lamda_HE[i], tota_spctr_HE[i])
        total_spctr_HE[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda_HE[i], tota_err_HE[i]**2)
        total_err_HE[i] = np.sqrt(interpol_error(ref_wave))
        
        total_lamda_HE[i] = ref_wave
        
        #Populating ANTARESS arrays
        #Normalized
        testE = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_norm_spctr_HE[i], cov = np.array([tota_norm_err_HE[i]**2]) , kind='cubic')

        total_norm_spctr_Antaress_HE[i] = testE[0]

        total_norm_err_Antaress_HE[i] = np.sqrt(testE[1][0])
        
        #Un-normalized
        testF = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_spctr_HE[i], cov = np.array([tota_err_HE[i]**2]) , kind='cubic')
        
        total_spctr_Antaress_HE[i] = testF[0]
        
        total_err_Antaress_HE[i] = np.sqrt(testF[1][0])
        
        testD = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_telluric_HE[i] , kind='cubic')
        
        total_telluric_HE[i] = testD

else:
    
    #Making the bins 
    min_lamda = max(tota_lamda[:, 0])
    max_lamda = min(tota_lamda[:, -1])
    
    ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda[0]))

    #Making the bins for Antaress
    preliminary_lamda=np.zeros((tota_lamda.shape[0], tota_lamda.shape[1]+1))
    
    for i in range(len(preliminary_lamda)):
        preliminary_lamda[i] = np.append(tota_lamda[i], tota_lamda[i][-1]+np.diff(tota_lamda[i])[-1])
    
    ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

    #Initializing variables
    total_lamda = np.zeros(tota_lamda.shape)

    total_norm_spctr = np.zeros(tota_norm_spctr.shape)

    total_norm_spctr_Antaress = np.zeros(tota_norm_spctr.shape)
        
    total_norm_err = np.zeros(tota_norm_err.shape)
        
    total_norm_err_Antaress = np.zeros(tota_norm_err.shape)

    total_spctr = np.zeros(tota_spctr.shape)
    
    total_spctr_Antaress = np.zeros(tota_spctr.shape)

    total_err = np.zeros(tota_err.shape)
    
    total_err_Antaress = np.zeros(tota_err.shape)
    
    total_telluric = np.zeros(tota_telluric.shape)
        
    for i in range(len(tota_lamda)):
        
        interpol_s = interp1d(tota_lamda[i], tota_norm_spctr[i])
        total_norm_spctr[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda[i], tota_norm_err[i]**2)
        total_norm_err[i] = np.sqrt(interpol_err(ref_wave))
        
        test3 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_norm_spctr[i], cov = np.array([tota_norm_err[i]**2]) , kind='cubic')
        
        total_norm_spctr_Antaress[i] = test3[0]
        
        total_norm_err_Antaress[i] = np.sqrt(test3[1][0])
                    
        test4 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_spctr[i], cov = np.array([tota_err[i]**2]) , kind='cubic')
        
        total_spctr_Antaress[i] = test4[0]
        
        total_err_Antaress[i] = np.sqrt(test4[1][0])
        
        test5 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_telluric[i], kind='cubic')
        
        total_telluric[i] = test5
        
        interpol_spctr = interp1d(tota_lamda[i], tota_spctr[i])
        total_spctr[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda[i], tota_err[i]**2)
        total_err[i] = np.sqrt(interpol_error(ref_wave))

        total_lamda[i] = ref_wave

## Comparing Interpolation Methods (Optional)

In [ ]:
h = 10

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):    
    
    fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2, sharex=True, figsize=[12, 8])
    ax2.plot(total_lamda_HA[h], np.abs((total_norm_spctr_Antaress_HA[h] - total_norm_spctr_HA[h])/total_norm_spctr_HA[h])*100, 'k.')
    ax2.set_xlabel('Wavelength ($\AA$)')
    ax2.set_ylabel('Fractional difference (%)')
    ax1.set_title('Interpolation Comparison - HA')
    ax1.errorbar(total_lamda_HA[h], total_norm_spctr_Antaress_HA[h], yerr=total_norm_err_Antaress_HA[h], fmt='b.', label='Antaress')
    ax1.errorbar(total_lamda_HA[h], total_norm_spctr_HA[h], yerr=total_norm_err_HA[h], fmt='r.', label='Basic')
    ax1.set_ylabel('Normalized Flux')
    ax1.legend()

    ax4.plot(total_lamda_HE[h], np.abs((total_norm_spctr_Antaress_HE[h] - total_norm_spctr_HE[h])/total_norm_spctr_HE[h])*100, 'k.')
    ax4.set_xlabel('Wavelength ($\AA$)')
    ax4.set_ylabel('Fractional difference (%)')
    ax3.set_title('Interpolation Comparison - HE')
    ax3.errorbar(total_lamda_HE[h], total_norm_spctr_Antaress_HE[h], yerr=total_norm_err_Antaress_HE[h], fmt='b.', label='Antaress')
    ax3.errorbar(total_lamda_HE[h], total_norm_spctr_HE[h], yerr=total_norm_err_HE[h], fmt='r.', label='Basic')
    ax3.set_ylabel('Normalized Flux')
    fig.subplots_adjust(hspace=0)

else:
    error_A = total_norm_err_Antaress[h]/total_norm_spctr[h]
    error_B = (total_norm_spctr_Antaress[h] * total_norm_err[h])/total_norm_spctr[h]**2
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=[14, 10])
    #ax2.errorbar(total_lamda[h], np.abs((total_norm_spctr_Antaress[h] - total_norm_spctr[h])/total_norm_spctr[h])*100, yerr=100*np.sqrt(error_A**2 + error_B**2), fmt='k.', alpha=0.1)
    ax2.plot(total_lamda[h], np.abs((total_norm_spctr_Antaress[h] - total_norm_spctr[h])/total_norm_spctr[h])*100, 'k.')
    ax2.set_xlabel('Wavelength ($\AA$)')
    ax2.set_ylabel('Fractional difference (%)')
    ax1.set_title('Interpolation Comparison')
    ax1.errorbar(total_lamda[h], total_norm_spctr_Antaress[h], yerr=total_norm_err_Antaress[h], fmt='b.', label='Antaress')
    ax1.errorbar(total_lamda[h], total_norm_spctr[h], yerr=total_norm_err[h], fmt='r.', label='Basic')
    ax1.set_ylabel('Normalized Flux')
    ax1.legend()
    fig.subplots_adjust(hspace=0)
plt.show()

## Continuum Portions used for Normalization and EW calculation (Optional)

In [ ]:
#for h in range(0, len(total_lamda), 20):
h=10

wav_low = 10750
wav_up = 10860

#Continuum Ranges
## Defining ranges for the continuum
wav_ranges = [(10804.8, 10805.9), (10807.4, 10808.5), (10825.5, 10826.5), (10839, 10840), (10842.4, 10843.1), (10850.2, 10851.2), (10856.5, 10857.9)]
##Retrieving the spectrum, error and wavelengths in these ranges
continuu_spctr = []
continuu_wav = []
continuu_err = []
for x in wav_ranges:
    contin_spctr = AF.bound(x[0], x[1], total_lamda[h], total_norm_spctr_Antaress[h])
    contin_err = AF.bound(x[0], x[1], total_lamda[h], total_norm_err_Antaress[h])
    contin_wav = AF.bound(x[0], x[1], total_lamda[h], total_lamda[h])
    continuu_spctr.append(contin_spctr)
    continuu_wav.append(contin_wav)
    continuu_err.append(contin_err)
continuum_spctr = np.array(list(itertools.chain.from_iterable(continuu_spctr)))
continuum_err = np.array(list(itertools.chain.from_iterable(continuu_err)))
continuum_wav = np.array(list(itertools.chain.from_iterable(continuu_wav)))

##Fitting the continuum range with  a 1st order polynomial 
p_continuum = np.poly1d(np.polyfit(continuum_wav, continuum_spctr, 1, w=1/continuum_err**2))

#Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14, 8], sharex=True)
ax1.errorbar(total_lamda[h], total_norm_spctr_Antaress[h], yerr=total_norm_err_Antaress[h], label='Data')
ax1.plot(total_lamda[h], p_continuum(total_lamda[h]), label='Polyn. model')
ax2.plot(total_lamda[h], total_telluric[h])
for i, x in enumerate(wav_ranges):
    ax1.axvline(x[0], color='r', linestyle='--')
    ax2.axvline(x[0], color='r', linestyle='--')
    ax1.axvline(x[1], color='r', linestyle='--')
    ax2.axvline(x[1], color='r', linestyle='--')
    if i==0:
        ax1.axvspan(x[0], x[1], color='r', alpha=0.3, label='Continuum Range')
    else:
        ax1.axvspan(x[0], x[1], color='r', alpha=0.3)
    ax2.axvspan(x[0], x[1], color='r', alpha=0.3)

ax1.axvline(10833.2, color='red', label='He triplet location')
ax1.axvline(10833.3, color='red')
ax1.axvline(10832.1, color='red')
ax1.axvline(10830.1, color='green')
ax1.axvline(10830.1, color='green', label='Si line location')
ax2.axvline(10833.2, color='red')
ax2.axvline(10833.3, color='red')
ax2.axvline(10832.1, color='red')
ax2.axvline(10830.1, color='green')
ax2.axvline(10830.1, color='green')
ax1.set_ylabel('Stellar Spectrum')
ax2.set_ylabel('Telluric Spectrum')
ax2.set_xlabel('Wavelength ($\AA$)')
ax1.set_xlim(wav_ranges[0][0]-5, wav_ranges[-1][1]+5)
ax1.legend()
fig.subplots_adjust(hspace=0)
plt.show()

# Main analysis

## Calculating the systematics

In [ ]:
reload(AF)
low = 10767
up = 10770

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    SNR_HA = AF.SNR_calculator(low, up, total_lamda_HA, total_spctr_Antaress_HA, total_err_Antaress_HA)
    #print(SNR_HA)

    SNR_HE = AF.SNR_calculator(low, up, total_lamda_HE, total_spctr_Antaress_HE, total_err_Antaress_HE)
    #print(SNR_HE)
    systematics_HA = total_SNR_HA - SNR_HA
    
    systematics_HE = total_SNR_HE - SNR_HE

else:
    SNR = AF.SNR_calculator(low, up, total_lamda, total_spctr_Antaress, total_err_Antaress)
    
    systematics = total_SNR - SNR
    #print(SNR)

## Fitting a gaussian to the He triplet line

### Test - Full line

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827.0
#lower_lim = 10830.5
upper_lim = 10834.1

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), 3, 15, 17, 2500, -500, -500, 10, -0.002]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit - Too loose priors
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Better priors - 1st round                                                      #100
#bounds_param_better = ((3, 3, 10, 10, 0, -5000, -5000, -50, -1), (7, 7, 20, 20, 10000, 5000, 5000, 50, 1))
#Better priors - 2nd round - Si temperature was really going to the bottom -> wants to go lower, made lower bound quite low, let's see what happens                                                 
bounds_param_better = ((3.9, 2, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005), (4.5, 3.5, 15.5, 17.1, 3470, 200, 170, 50, 0))

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)
    
    better_theta, better_err_theta, better_lmfit_thetas, better_err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

In [ ]:
fig, ax = plt.subplots(5, 2, sharex = True, figsize = [10, 12])

param = theta
param_err = err_theta
param_lmfit = lmfit_thetas
param_lmfit_err = err_lmfit_thetas
time = date
axs = ax.ravel()
for i in range(len(axs)):
    if i==1:
        axs[i].plot(date, param[:, i], 'b.', label='Curve fit')
        axs[i].plot(date, param_lmfit[:, i], 'r.', label='Lmfit')
    else:
        axs[i].plot(date, param[:, i], 'b.')
        axs[i].plot(date, param_lmfit[:, i], 'r.')
    if i<9:
        axs[i].set_ylabel(param_name[i])
    else:
        axs[i].set_ylabel('Eq. width')

axs[8].set_xlabel('Time (MJD)')
axs[9].set_xlabel('Time (MJD)')

axs[1].legend()
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 2, sharex = True, figsize = [7, 8])

param = better_theta
param_err = better_err_theta
param_lmfit = better_lmfit_thetas
param_lmfit_err = better_err_lmfit_thetas
time = date
axs = ax.ravel()
for i in range(len(axs)):
    if i==1:
        axs[i].plot(date, param[:, i], 'b.', label='Curve fit')
        axs[i].plot(date, param_lmfit[:, i], 'r.', label='Lmfit')
    else:
        axs[i].plot(date, param[:, i], 'b.')
        axs[i].plot(date, param_lmfit[:, i], 'r.')
    if i<9:
        axs[i].set_ylabel(param_name[i])
    else:
        axs[i].set_ylabel('Eq. width')

axs[8].set_xlabel('Time (MJD)')
axs[9].set_xlabel('Time (MJD)')

axs[1].legend()
plt.subplots_adjust(hspace=0)
plt.show()

### Test - Left wing

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10827.0, 10829.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Right wing

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10830.6, 10831.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

#Temperature - 1000 - 10,000,000
#0 - 100 damping parameter (see Wikipedia)
#Density 10^8 - 10^20
#Look into the priors yourself 

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Both wings

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10827.0, 10829.5), (10830.6, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Voigt core

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, 0, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Gauss core

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, -np.inf, -np.inf, 0, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si_Gauss(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### More testing

In [ ]:
%matplotlib inline

h=50
total_range = [10827, 10834.1]
core_range = [10829.78, 10830.38]
left_wing_range = [10827, 10829.7]
right_wing_range = [10830.45, 10834.1]
bound_lamda = AF.bound(total_range[0], total_range[1], total_lamda[h], total_lamda[h])
bound_flux = AF.bound(total_range[0], total_range[1], total_lamda[h], total_norm_spctr_Antaress[h])
#For plotting purposes#
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=[12, 4])
core_lamda = AF.bound(core_range[0], core_range[1], total_lamda[h], total_lamda[h]) 
core_flux = AF.bound(core_range[0], core_range[1], total_lamda[h], total_norm_spctr_Antaress[h]) 
left_wing_lamda = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[h], total_lamda[h])
left_wing_flux = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[h], total_norm_spctr_Antaress[h]) 
right_wing_lamda = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[h], total_lamda[h]) 
right_wing_flux = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[h], total_norm_spctr_Antaress[h])

ax1.plot(left_wing_lamda, left_wing_flux, 'b')
ax1.plot(core_lamda, core_flux, 'r')
ax1.plot(right_wing_lamda, right_wing_flux, 'b')

##

wing_f = np.zeros(bound_lamda.shape)
core_f = np.zeros(bound_lamda.shape)

wing_lambda = np.append(left_wing_lamda, right_wing_lamda)
core_lambda = np.copy(core_lamda)
print(len(bound_lamda))
for i, x in enumerate(bound_lamda):
    if left_wing_range[0] < x < left_wing_range[1] or right_wing_range[0] < x < right_wing_range[1]:
        wing_f[i] = 1.
    elif core_range[0] < x < core_range[1]:
        core_f[i] = 1.
##

ax1.plot(bound_lamda, wing_f, 'b.')
ax1.plot(bound_lamda, core_f, 'r.')

'''new_wing_lambda = np.copy(bound_lamda)
new_core_lambda = np.copy(bound_lamda)
for i, x in enumerate(len(bound_param)):
    if new_wing_lambda[i] not in wing_lambda:
        new_wing_lambda = 0
    if new_core_lambda[i] not in core_lambda:
        new_core_lambda = 0'''
    
    
model_wings = spec_mod.spectrum_he_and_Si(bound_lamda, wing_f,  10**4.4, 10**5.5, 
                           10**15, 10**16.15, vsini=None, medium='VACUUM', Si_delta_damping=2500, RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)

model_core = spec_mod.spectrum_he_and_Si_Gauss(bound_lamda, core_f,  10**4, 10**5, 
                           10**14, 10**16.1, vsini=None, medium='VACUUM', RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, Si_delta_damping=0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)

model_Voigt = spec_mod.spectrum_he_and_Si(bound_lamda, np.ones(bound_lamda.shape),  10**4.5, 10**4.7, 
                           10**15.1, 10**16.27, vsini=None, medium='VACUUM', Si_delta_damping=2500, RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)


ax2.plot(bound_lamda, bound_flux)
ax2.plot(bound_lamda, model_wings, 'k.-',label='Complicated model', alpha=0.3)
ax2.plot(bound_lamda, model_core, 'k--', alpha=0.3)
ax2.plot(bound_lamda, model_core+model_wings, 'k.')

ax3.plot(bound_lamda, bound_flux)
ax3.plot(bound_lamda, model_Voigt, 'g.-')
plt.subplots_adjust(wspace=0)
ax1.set_ylabel('Normalized Flux')
ax1.set_xlabel('Wavlength ($\AA$)')
ax2.set_xlabel('Wavlength ($\AA$)')
ax3.set_xlabel('Wavlength ($\AA$)')
ax1.set_title('Study range')
ax2.set_title('Cut up fit')
ax3.set_title('Together fit')


ax1.set_ylim(0.35, 1.1)


### Fitting wings with Voigt and core with Gauss

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

core_range = [10829.78, 10830.38]
left_wing_range = [10827, 10829.78]
right_wing_range = [10830.38, 10834.1]

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1

ref_Si = AF.air2vac(10827.091)
c_light_m=299792458.

#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]

#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.8, 14.7, 17, 2500, -700, -500, 10, -0.002]

#guess_params with variable range
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.78, 10830.38]
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.5, 10830.5]
guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.5]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit

#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 

#Bounds for lmfit - Too loose priors
#bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))

#Better priors - 1st round                                                      #100
#bounds_param_better = ((3.9, 3, 10, 10, 0, -5000, -5000, 0, -1), (7, 7, 14.8, 20, 10000, 5000, 5000, 50, 1))

#Better priors - 2nd round
#bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 10829, 10830.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 10829.85, 10831))
bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 0.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 1))

#Better priors - 2nd round - with variable range

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    core_lamda = AF.bound(core_range[0], core_range[1], total_lamda[0], total_lamda[0]) 
    left_wing_lamda = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[0], total_lamda[0])
    right_wing_lamda = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[0], total_lamda[0]) 


    wing_lambda = np.append(left_wing_lamda, right_wing_lamda)
    core_lambda = np.copy(core_lamda)

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges):
        #Describing the continuum with a polynomial
        poly = A+B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        lamda_offset_Si = (ref_Si * RV_offset_Si)/c_light_m
        for i, j in enumerate(x):
            if lower_lim <= j <= (ref_Si + lamda_offset_Si - ranges) or (ref_Si + lamda_offset_Si + ranges) <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif (ref_Si + lamda_offset_Si - ranges) <= j <= (ref_Si + lamda_offset_Si + ranges):
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core     
        return full_model


    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'ranges']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)


### Testing

In [ ]:
from scipy import optimize

z = 5
def toy_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges):

        lamda_offset_Si = (ref_Si * RV_offset_Si)/c_light_m

        cond_list = [np.logical_or(np.logical_and(lower_lim <= x, x <= (ref_Si + lamda_offset_Si - ranges)), np.logical_and((ref_Si + lamda_offset_Si + ranges) <= x, x <= upper_lim)), np.logical_and((ref_Si + lamda_offset_Si - ranges) <= x, x <= (ref_Si + lamda_offset_Si + ranges))]

        func_list = [lambda x: spec_mod.spectrum_he_and_Si(x, A+B*x,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power), 
                    lambda x: spec_mod.spectrum_he_and_Si_Gauss(x, A+B*x,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)]

        return np.piecewise(x, cond_list, func_list)

    
toy_x = np.linspace(10820, 10840, 100)
toy_y = toy_model(toy_x, 4.3422, 4.9999, 14.927, 16.094, 4839, -1329, -560, 11.66, -0.0009867, 0.3)
plt.plot(total_lamda[z], total_norm_spctr_Antaress[z], '.')
plt.plot(toy_x, toy_y, 'r')
plt.xlim(10827, 10834.1)

bou_x = AF.bound(lower_lim, upper_lim, total_lamda[z], total_lamda[z])
bou_y = AF.bound(lower_lim, upper_lim, total_lamda[z], total_norm_spctr_Antaress[z])
bou_y_err = AF.bound(lower_lim, upper_lim, total_lamda[z], total_norm_err_Antaress[z])

guess = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.3]
bou = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 0.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 1))
para, cov = curve_fit(toy_model, bou_x, bou_y, sigma = bou_y_err, p0 = guess, bounds = bou)

plt.plot(toy_x, toy_model(toy_x, *para), 'k')
plt.show()

print(para)


def optimizeFunc(p, extra):
    log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges = p
    x, y = extra
    return toy_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges)-y


fit, ier = optimize.leastsq(optimizeFunc, guess, [bou_x, bou_y])
print(fit)








### Fitting a gaussian to the He triplet line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)
lower_lim = 10831.5
upper_lim = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 
guess_params = [10, 10833.3, 0.3, -10]
bounds_param = ((-np.inf, 10833.2, 0, -np.inf),(np.inf, 10833.4, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_rassine_HA, err_theta_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_rassine_HE, err_theta_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    theta_rassine, err_theta_rassine = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, lower_lim, upper_lim, guess_params, 
                                         bounds_param, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, 'r', wav_ranges)

    theta_rassine[:, 0] = np.abs(theta_rassine[:, 0])
    

### Fitting a gaussian to the He triplet line - Normal data + Antaress

In [ ]:
reload(AF)
lower_lim = 10831.5
upper_lim = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 
guess_params = [10, 10833.3, 0.3, 0.001, 0.001, -10]
bounds_param = ((-np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf),(np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Antaress_HA, err_theta_Antaress_HA = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, guess_params, 
                                               bounds_param, total_lamda_HA, total_norm_spctr_Antaress_HA, 
                                               total_norm_err_Antaress_HA, 'r', wav_ranges)

    theta_Antaress_HE, err_theta_Antaress_HE = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, guess_params, 
                                               bounds_param, total_lamda_HE, total_norm_spctr_Antaress_HE, 
                                               total_norm_err_Antaress_HE, 'k', wav_ranges)
    
    theta_Antaress_HA[:, 0] = np.abs(theta_Antaress_HA[:, 0])
    theta_Antaress_HE[:, 0] = np.abs(theta_Antaress_HE[:, 0])
#If there is one mode of observation
else:
    theta_Antaress, err_theta_Antaress = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, lower_lim, upper_lim, guess_params, 
                                         bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges)

    theta_Antaress[:, 0] = np.abs(theta_Antaress[:, 0])
    

## Plotting the positions, depths and widths of the fitted Gaussian lines

### Plotting the test

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59970

depth_index = 1 #actually temperature
position_index = 0 #actually log density
width_index = 2 #actually the offset
ew_index = -1 #actually the ew

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_HA
    err_in_question = err_theta_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_HE
    err_in_question = err_theta_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    '''
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta
    err_in_question = err_theta
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = theta_in_question[:, position_index]
    new_pos_err = err_in_question[:, position_index]
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, theta_in_question[:, position_index], yerr=err_in_question[:, position_index], fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, binned_pos, yerr=binned_pos_err, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax1.plot(date_in_question-ref_time, theta_in_question[:, depth_index], '.', alpha=0.2)
    #ax1.plot(binned_date-ref_time, binned_depth-15, 'r.', markersize=10)
    #ax1.plot(binned_date-ref_time, binned_depth, 'r.', markersize=10)
    
    ax2.plot(date_in_question-ref_time, theta_in_question[:, position_index], '.', alpha=0.2)
    #ax2.plot(binned_date-ref_time, binned_pos-20000, 'r.', markersize=10)
    ax2.plot(binned_date-ref_time, binned_pos, 'r.', markersize=10)
    
    ax3.plot(date_in_question-ref_time, theta_in_question[:, width_index], '.', alpha=0.2)
    ax3.plot(binned_date-ref_time, binned_width, 'r.', markersize=10)
    
    ax4.plot(date_in_question-ref_time, theta_in_question[:, ew_index], '.', alpha=0.2)
    ax4.plot(binned_date-ref_time, binned_ew, 'r.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Density ($m^{-3}$)', fontsize=14)
    ax2.set_ylabel('Temperature (K)', fontsize=14)
    ax3.set_ylabel('Offset (flux)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit.pdf')
    plt.show()
    '''

    T_He_index = 0 #actually temperature of He
    T_Si_index = 1 #actually temperature of Si
    n_He_index = 2 #actually density of He
    n_Si_index = 3 #actually density of Si
    d_Si_index = 4 # actually damping parameter of Si
    ew_index = -1 #actually the ew

    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots (3, 2, sharex=True, figsize=[12, 10])

    theta_in_question = lmfit_thetas
    err_in_question = np.ones(lmfit_thetas.shape)*0.00001
    date_in_question = date
    
    #Getting the scatter
    avg_T_He = np.average(theta_in_question[:, T_He_index])
    std_T_He = np.sqrt(np.average((theta_in_question[:, T_He_index] - avg_T_He)**2))
    avg_T_Si = np.average(theta_in_question[:, T_Si_index])
    std_T_Si = np.sqrt(np.average((theta_in_question[:, T_Si_index] - avg_T_Si)**2))
    avg_n_He = np.average(theta_in_question[:, n_He_index])
    std_n_He = np.sqrt(np.average((theta_in_question[:, n_He_index] - avg_n_He)**2))
    avg_n_Si = np.average(theta_in_question[:, n_Si_index])
    std_n_Si = np.sqrt(np.average((theta_in_question[:, n_Si_index] - avg_n_Si)**2))
    avg_d_Si = np.average(theta_in_question[:, d_Si_index])
    std_d_Si = np.sqrt(np.average((theta_in_question[:, d_Si_index] - avg_d_Si)**2))   
    avg_ew = np.average(theta_in_question[:, ew_index])
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_T_He = ss.binned_statistic(date_in_question, theta_in_question[:, T_He_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_n_He = ss.binned_statistic(date_in_question, theta_in_question[:, n_He_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_T_Si = ss.binned_statistic(date_in_question, theta_in_question[:, T_Si_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_n_Si = ss.binned_statistic(date_in_question, theta_in_question[:, n_Si_index] , statistic=np.ma.median, bins=num_bins)[0]
    binned_d_Si = ss.binned_statistic(date_in_question, theta_in_question[:, d_Si_index] , statistic=np.ma.median, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]

    ax1.plot(date_in_question-ref_time, theta_in_question[:, T_He_index], '.', alpha=0.2)
    ax1.plot(binned_date-ref_time, binned_T_He, 'r.', markersize=10)
    ax1twin = ax1.twinx()
    ax1twin.plot(date_in_question-ref_time, 10**theta_in_question[:, T_He_index], '.', alpha=0)
    
    ax2.plot(date_in_question-ref_time, theta_in_question[:, T_Si_index], '.', alpha=0.2)
    ax2.plot(binned_date-ref_time, binned_T_Si, 'r.', markersize=10)
    ax2twin = ax2.twinx()
    ax2twin.plot(date_in_question-ref_time, 10**theta_in_question[:, T_Si_index], '.', alpha=0)

    ax3.plot(date_in_question-ref_time, theta_in_question[:, n_He_index], '.', alpha=0.2)
    ax3.plot(binned_date-ref_time, binned_n_He, 'r.', markersize=10)
 
    ax4.plot(date_in_question-ref_time, theta_in_question[:, n_Si_index], '.', alpha=0.2)
    ax4.plot(binned_date-ref_time, binned_n_Si, 'r.', markersize=10)

    ax5.plot(date_in_question-ref_time, theta_in_question[:, d_Si_index], '.', alpha=0.2)
    ax5.plot(binned_date-ref_time, binned_d_Si, 'r.', markersize=10)
    
    ax6.plot(date_in_question-ref_time, theta_in_question[:, ew_index], '.', alpha=0.2)
    ax6.plot(binned_date-ref_time, binned_ew, 'r.', markersize=10)
    
    ax5.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax6.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Log Temperature He (K)', fontsize=14)
    ax2.set_ylabel('Log Temperature Si (K)', fontsize=14)
    ax3.set_ylabel('Log Density He ($m^{-3}$)', fontsize=14)
    ax4.set_ylabel('Log Density Si ($m^{-3}$)', fontsize=14)
    ax5.set_ylabel('Damping Si', fontsize=14)
    ax6.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='y', labelsize=14)
    ax4.tick_params(axis='y', labelsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_T_He
    textstr2 = '$\sigma$ = %.3f'%std_T_Si
    textstr3 = '$\sigma$ = %.3f'%std_n_He
    textstr4 = '$\sigma$ = %.3f'%std_n_Si
    textstr5 = '$\sigma$ = %.3f'%std_d_Si
    textstr6 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax5.text(0.67, 0.92, textstr5, transform=ax5.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax6.text(0.67, 0.92, textstr6, transform=ax6.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit.pdf')
    plt.show()
    

### Plotting the positions, depths and widths of the fitted Gaussian lines - Rassine data + Basic Interpolation

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine_HA
    err_in_question = err_theta_rassine_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine_HE
    err_in_question = err_theta_rassine_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine
    err_in_question = err_theta_rassine
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian lines - Normal data + Antaress

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress_HA
    err_in_question = err_theta_Antaress_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress_HE
    err_in_question = err_theta_Antaress_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress
    err_in_question = err_theta_Antaress
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian lines - Overplotted (Rassine + Antaress)

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress_HA[:, depth_index], weights=1/err_theta_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress_HA[:, width_index], weights=1/err_theta_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Antaress_HA[:, ew_index], weights=1/err_theta_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Antaress_HA[:, depth_index], yerr=err_theta_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Antaress_HA[:, width_index], yerr=err_theta_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Antaress_HA[:, ew_index], yerr=err_theta_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine_HA[:, depth_index], weights=1/err_theta_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine_HA[:, depth_index]**2))
    new_pos = (theta_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine_HA[:, width_index], weights=1/err_theta_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_rassine_HA[:, ew_index], weights=1/err_theta_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, theta_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_rassine_HA[:, depth_index], yerr=err_theta_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_rassine_HA[:, width_index], yerr=err_theta_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_rassine_HA[:, ew_index], yerr=err_theta_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress_HE[:, depth_index], weights=1/err_theta_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress_HE[:, width_index], weights=1/err_theta_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Antaress_HE[:, ew_index], weights=1/err_theta_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Antaress_HE[:, depth_index], yerr=err_theta_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Antaress_HE[:, width_index], yerr=err_theta_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Antaress_HE[:, ew_index], yerr=err_theta_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine_HE[:, depth_index], weights=1/err_theta_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine_HE[:, depth_index]**2))
    new_pos = (theta_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine_HE[:, width_index], weights=1/err_theta_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_rassine_HE[:, ew_index], weights=1/err_theta_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, theta_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_rassine_HE[:, depth_index], yerr=err_theta_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_rassine_HE[:, width_index], yerr=err_theta_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_rassine_HE[:, ew_index], yerr=err_theta_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress[:, depth_index], weights=1/err_theta_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress[:, depth_index]**2))
    new_pos = (theta_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress[:, width_index], weights=1/err_theta_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Antaress[:, ew_index], weights=1/err_theta_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Antaress[:, depth_index], yerr=err_theta_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (theta_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Antaress[:, width_index], yerr=err_theta_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Antaress[:, ew_index], yerr=err_theta_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine[:, depth_index], weights=1/err_theta_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine[:, depth_index]**2))
    new_pos = (theta_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine[:, width_index], weights=1/err_theta_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_rassine[:, width_index]**2))
    avg_ew = np.average(theta_rassine[:, ew_index], weights=1/err_theta_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, theta_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_rassine[:, depth_index], yerr=err_theta_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (theta_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_rassine[:, width_index], yerr=err_theta_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_rassine[:, ew_index], yerr=err_theta_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot.pdf')
    plt.show()

## Fitting a Voigt profile to the Si line

### Fitting a Voigt profile to the Si line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)

lower_l = 10828
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Si_rassine_HA, err_theta_Si_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HA, rassine_total_norm_spctr_HA, rassine_total_norm_err_HA, 'r', wav_ranges)
    
    theta_Si_rassine_HE, err_theta_Si_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HE, rassine_total_norm_spctr_HE, rassine_total_norm_err_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    theta_Si_rassine, err_theta_Si_rassine = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda, rassine_total_norm_spctr, rassine_total_norm_err, 'r', wav_ranges)
    

### Fitting a Voigt profile to the Si line - Normal data + Antaress

In [ ]:
reload(AF)

lower_l = 10828
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, 0.001, 0.001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Si_Antaress_HA, err_theta_Si_Antaress_HA = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HA, total_norm_spctr_Antaress_HA, total_norm_err_Antaress_HA, 'r', wav_ranges)
    
    theta_Si_Antaress_HE, err_theta_Si_Antaress_HE = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HE, total_norm_spctr_Antaress_HE, total_norm_err_Antaress_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    theta_Si_Antaress, err_theta_Si_Antaress = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  total_lamda, total_norm_spctr_Antaress, total_norm_err_Antaress, 'r', wav_ranges)


## Fitting a Gaussian profile to the Si line

### Fitting a Gaussian profile to the Si line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)

#lower_l = 10827.5
#upper_l = 10832
lower_l = 10828.7
upper_l = 10831.5
guess_para = [10, 10830, 0.3, -10]
bound = ((-np.inf, 10828, 0, -np.inf),(np.inf, 10832, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    thetaG_Si_rassine_HA, err_thetaG_Si_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HA, rassine_total_norm_spctr_HA, rassine_total_norm_err_HA, 'r', wav_ranges)
    
    thetaG_Si_rassine_HE, err_thetaG_Si_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HE, rassine_total_norm_spctr_HE, rassine_total_norm_err_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    thetaG_Si_rassine, err_thetaG_Si_rassine = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda, rassine_total_norm_spctr, rassine_total_norm_err, 'r', wav_ranges)

### Fitting a Gaussian profile to the Si line - Normal data + Antaress

In [ ]:
reload(AF)

lower_l = 10827.5
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, 0.0001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    thetaG_Si_Antaress_HA, err_thetaG_Si_Antaress_HA = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HA, total_norm_spctr_Antaress_HA, total_norm_err_Antaress_HA, 'r', wav_ranges)
    
    thetaG_Si_Antaress_HE, err_thetaG_Si_Antaress_HE = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HE, total_norm_spctr_Antaress_HE, total_norm_err_Antaress_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    thetaG_Si_Antaress, err_thetaG_Si_Antaress = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  total_lamda, total_norm_spctr_Antaress, total_norm_err_Antaress, 'r', wav_ranges)

## Plotting the positions, depths and widths of the fitted Voigt profile for Si

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Rassine data + Basic Interpolation

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HA[:, depth_index], weights=1/err_theta_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HA[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HA[:, width_index], weights=1/err_theta_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HA[:, ew_index], weights=1/err_theta_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, depth_index], yerr=err_theta_Si_rassine_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (theta_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, width_index], yerr=err_theta_Si_rassine_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, ew_index], yerr=err_theta_Si_rassine_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HE[:, depth_index], weights=1/err_theta_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HE[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HE[:, width_index], weights=1/err_theta_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HE[:, ew_index], weights=1/err_theta_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, depth_index], yerr=err_theta_Si_rassine_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (theta_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, width_index], yerr=err_theta_Si_rassine_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, ew_index], yerr=err_theta_Si_rassine_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine[:, depth_index], weights=1/err_theta_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine[:, depth_index]**2))
    new_pos = (theta_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine[:, width_index], weights=1/err_theta_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine[:, ew_index], weights=1/err_theta_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_rassine[:, depth_index], yerr=err_theta_Si_rassine[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (theta_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_rassine[:, width_index], yerr=err_theta_Si_rassine[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_rassine[:, ew_index], yerr=err_theta_Si_rassine[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Normal data + Antaress

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HA[:, depth_index], weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HA[:, width_index], weights=1/err_theta_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HA[:, ew_index], weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, depth_index], yerr=err_theta_Si_Antaress_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (theta_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, width_index], yerr=err_theta_Si_Antaress_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, ew_index], yerr=err_theta_Si_Antaress_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HE[:, depth_index], weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HE[:, width_index], weights=1/err_theta_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HE[:, ew_index], weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, depth_index], yerr=err_theta_Si_Antaress_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (theta_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, width_index], yerr=err_theta_Si_Antaress_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, ew_index], yerr=err_theta_Si_Antaress_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress[:, depth_index], weights=1/err_theta_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress[:, depth_index]**2))
    new_pos = (theta_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress[:, width_index], weights=1/err_theta_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress[:, ew_index], weights=1/err_theta_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_Antaress[:, depth_index], yerr=err_theta_Si_Antaress[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (theta_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_Antaress[:, width_index], yerr=err_theta_Si_Antaress[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_Antaress[:, ew_index], yerr=err_theta_Si_Antaress[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Overplot (Rassine + Antaress)

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HA[:, depth_index], weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HA[:, width_index], weights=1/err_theta_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HA[:, ew_index], weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, depth_index], yerr=err_theta_Si_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, width_index], yerr=err_theta_Si_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, ew_index], yerr=err_theta_Si_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HA[:, depth_index], weights=1/err_theta_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HA[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HA[:, width_index], weights=1/err_theta_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HA[:, ew_index], weights=1/err_theta_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, depth_index], yerr=err_theta_Si_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, width_index], yerr=err_theta_Si_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, ew_index], yerr=err_theta_Si_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HE[:, depth_index], weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HE[:, width_index], weights=1/err_theta_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HE[:, ew_index], weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, depth_index], yerr=err_theta_Si_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, width_index], yerr=err_theta_Si_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, ew_index], yerr=err_theta_Si_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HE[:, depth_index], weights=1/err_theta_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HE[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HE[:, width_index], weights=1/err_theta_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HE[:, ew_index], weights=1/err_theta_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, depth_index], yerr=err_theta_Si_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, width_index], yerr=err_theta_Si_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, ew_index], yerr=err_theta_Si_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress[:, depth_index], weights=1/err_theta_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress[:, depth_index]**2))
    new_pos = (theta_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress[:, width_index], weights=1/err_theta_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress[:, ew_index], weights=1/err_theta_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_Antaress[:, depth_index], yerr=err_theta_Si_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (theta_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_Antaress[:, width_index], yerr=err_theta_Si_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_Antaress[:, ew_index], yerr=err_theta_Si_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine[:, depth_index], weights=1/err_theta_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine[:, depth_index]**2))
    new_pos = (theta_Si_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine[:, width_index], weights=1/err_theta_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine[:, ew_index], weights=1/err_theta_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_rassine[:, depth_index], yerr=err_theta_Si_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (theta_Si_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_rassine[:, width_index], yerr=err_theta_Si_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_rassine[:, ew_index], yerr=err_theta_Si_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot.pdf')
    plt.show()

## Plotting the positions, depths and widths of the fitted Gaussian profile for Si

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Rassine data + Basic Interpolation

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HA[:, depth_index], weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HA[:, width_index], weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HA[:, ew_index], weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, depth_index], yerr=err_thetaG_Si_rassine_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, width_index], yerr=err_thetaG_Si_rassine_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, ew_index], yerr=err_thetaG_Si_rassine_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HE[:, depth_index], weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HE[:, width_index], weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HE[:, ew_index], weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, depth_index], yerr=err_thetaG_Si_rassine_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, width_index], yerr=err_thetaG_Si_rassine_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, ew_index], yerr=err_thetaG_Si_rassine_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine[:, depth_index], weights=1/err_thetaG_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine[:, width_index], weights=1/err_thetaG_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine[:, ew_index], weights=1/err_thetaG_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_rassine[:, depth_index], yerr=err_thetaG_Si_rassine[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (thetaG_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_rassine[:, width_index], yerr=err_thetaG_Si_rassine[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_rassine[:, ew_index], yerr=err_thetaG_Si_rassine[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Normal data + Antaress

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HA[:, depth_index], weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HA[:, width_index], weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HA[:, ew_index], weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, depth_index], yerr=err_thetaG_Si_Antaress_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, width_index], yerr=err_thetaG_Si_Antaress_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, ew_index], yerr=err_thetaG_Si_Antaress_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HE[:, depth_index], weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HE[:, width_index], weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HE[:, ew_index], weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, depth_index], yerr=err_thetaG_Si_Antaress_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, width_index], yerr=err_thetaG_Si_Antaress_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, ew_index], yerr=err_thetaG_Si_Antaress_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress[:, depth_index], weights=1/err_thetaG_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress[:, width_index], weights=1/err_thetaG_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress[:, ew_index], weights=1/err_thetaG_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_Antaress[:, depth_index], yerr=err_thetaG_Si_Antaress[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (thetaG_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_Antaress[:, width_index], yerr=err_thetaG_Si_Antaress[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_Antaress[:, ew_index], yerr=err_thetaG_Si_Antaress[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Overplot (Rassine + Antaress)

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HA[:, depth_index], weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HA[:, width_index], weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HA[:, ew_index], weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, depth_index], yerr=err_thetaG_Si_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, width_index], yerr=err_thetaG_Si_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, ew_index], yerr=err_thetaG_Si_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HA[:, depth_index], weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HA[:, width_index], weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HA[:, ew_index], weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, depth_index], yerr=err_thetaG_Si_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, width_index], yerr=err_thetaG_Si_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, ew_index], yerr=err_thetaG_Si_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HE[:, depth_index], weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HE[:, width_index], weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HE[:, ew_index], weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, depth_index], yerr=err_thetaG_Si_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, width_index], yerr=err_thetaG_Si_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, ew_index], yerr=err_thetaG_Si_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HE[:, depth_index], weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HE[:, width_index], weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HE[:, ew_index], weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, depth_index], yerr=err_thetaG_Si_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, width_index], yerr=err_thetaG_Si_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, ew_index], yerr=err_thetaG_Si_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress[:, depth_index], weights=1/err_thetaG_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress[:, width_index], weights=1/err_thetaG_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress[:, ew_index], weights=1/err_thetaG_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_Antaress[:, depth_index], yerr=err_thetaG_Si_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (thetaG_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_Antaress[:, width_index], yerr=err_thetaG_Si_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_Antaress[:, ew_index], yerr=err_thetaG_Si_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine[:, depth_index], weights=1/err_thetaG_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine[:, width_index], weights=1/err_thetaG_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine[:, ew_index], weights=1/err_thetaG_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_rassine[:, depth_index], yerr=err_thetaG_Si_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (thetaG_Si_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_rassine[:, width_index], yerr=err_thetaG_Si_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_rassine[:, ew_index], yerr=err_thetaG_Si_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot.pdf')
    plt.show()

## Fitting the line in the telluric spectra around the He triplet

In [ ]:
reload(AF)

lower_lim_T = 10832.5
upper_lim_T = 10834.1
wav_ranges_T = [(10758.2, 10762.4), (10778.2, 10781.2), (10804.8, 10805.7), (10807.4, 10808.4), (10838.7, 10839.7), (10856.7, 10858.7)]
guess_params_T = [0.01, 10833.3, 0.1, -0.1, 0.0001, -10]
bounds_param_T = ((-np.inf, 10832.9, 0, -np.inf, -np.inf, -np.inf),(np.inf, 10833.5, np.inf, np.inf, np.inf, np.inf))

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        
    theta_T_HA, err_theta_T_HA = AF.fit_spctr_line(AF.new_best_model, lower_lim_T, upper_lim_T, guess_params_T, 
                                               bounds_param_T, tota_lamda_HA, tota_telluric_HA, [], 'r', wav_ranges_T)

    theta_T_HE, err_theta_T_HE = AF.fit_spctr_line(AF.new_best_model, lower_lim_T, upper_lim_T, guess_params_T, 
                                               bounds_param_T, tota_lamda_HE, tota_telluric_HE, [], 'k', wav_ranges_T)

#If there is one mode of observation
else:
    theta_T, err_theta_T = AF.fit_spctr_line(AF.new_best_model, lower_lim_T, upper_lim_T, lower_lim_T, upper_lim_T, guess_params_T, 
                                               bounds_param_T, total_lamda, total_telluric, [], 'r', wav_ranges_T)      


## Plotting the time series of the position, width and depth of this line

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_T = 10833.3145 #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_T_HA[:, depth_index], weights=1/err_theta_T_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_T_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_T_HA[:, depth_index]**2))
    new_pos = (theta_T_HA[:, position_index]-ref_T)*3e8/ref_T
    new_pos_err = err_theta_T_HA[:, position_index]*3e8/ref_T
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_T_HA[:, width_index], weights=1/err_theta_T_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_T_HA[:, width_index] - avg_width)**2, weights=1/err_theta_T_HA[:, width_index]**2))
    avg_ew = np.average(theta_T_HA[:, ew_index], weights=1/err_theta_T_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_T_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_T_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_T_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_T_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_T_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_T_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_T_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_T_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_T_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_T_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_T_HA[:, depth_index], yerr=err_theta_T_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (theta_T_HA[:, position_index]-ref_T)*3e8/ref_T, yerr=err_theta_T_HA[:, position_index]*3e8/ref_T, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_T)*3e8/ref_T, yerr=binned_pos_err*3e8/ref_T, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_T_HA[:, width_index], yerr=err_theta_T_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_T_HA[:, ew_index], yerr=err_theta_T_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Gaussian_Fit_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_T_HE[:, depth_index], weights=1/err_theta_T_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_T_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_T_HE[:, depth_index]**2))
    new_pos = (theta_T_HE[:, position_index]-ref_T)*3e8/ref_T
    new_pos_err = err_theta_T_HE[:, position_index]*3e8/ref_T
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_T_HE[:, width_index], weights=1/err_theta_T_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_T_HE[:, width_index] - avg_width)**2, weights=1/err_theta_T_HE[:, width_index]**2))
    avg_ew = np.average(theta_T_HE[:, ew_index], weights=1/err_theta_T_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_T_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_T_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_T_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_T_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_T_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_T_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_T_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_T_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_T_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_T_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_T_HE[:, depth_index], yerr=err_theta_T_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (theta_T_HE[:, position_index]-ref_T)*3e8/ref_T, yerr=err_theta_T_HE[:, position_index]*3e8/ref_T, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_T)*3e8/ref_T, yerr=binned_pos_err*3e8/ref_T, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_T_HE[:, width_index], yerr=err_theta_T_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_T_HE[:, ew_index], yerr=err_theta_T_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Gaussian_Fit_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_T[:, depth_index], weights=1/err_theta_T[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_T[:, depth_index] - avg_depth)**2, weights=1/err_theta_T[:, depth_index]**2))
    new_pos = (theta_T[:, position_index]-ref_T)*3e8/ref_T
    new_pos_err = err_theta_T[:, position_index]*3e8/ref_T
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_T[:, width_index], weights=1/err_theta_T[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_T[:, width_index] - avg_width)**2, weights=1/err_theta_T[:, width_index]**2))
    avg_ew = np.average(theta_T[:, ew_index], weights=1/err_theta_T[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_T[:, ew_index] - avg_ew)**2, weights=1/err_theta_T[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_T[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_T[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_T[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_T[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_T[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_T[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_T[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_T[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_T[:, depth_index], yerr=err_theta_T[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (theta_T[:, position_index]-ref_T)*3e8/ref_T, yerr=err_theta_T[:, position_index]*3e8/ref_T, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_T)*3e8/ref_T, yerr=binned_pos_err*3e8/ref_T, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_T[:, width_index], yerr=err_theta_T[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_T[:, ew_index], yerr=err_theta_T[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Gaussian_Fit.pdf')
    plt.show()

# Correlation Studies

## Correlation between He triplet line and CCF RV

In [ ]:
reload(AF)

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

indexes = [0, 1, 2, -1]

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_RV_HA, total_RV_err_HA, indexes, ref_He, 'CCF RV (km/s)',
                         'Correlation_RV', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_RV_HE, total_RV_err_HE, indexes, ref_He, 'CCF RV (km/s)',
                         'Correlation_RV', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_RV, total_RV_err, indexes, ref_He, 'CCF RV (km/s)', 
                             'Correlation_RV', False)

## Correlation with H2O column density

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_H2O_HA/total_AIRM_HA, total_H2O_err_HA/total_AIRM_HA, indexes, 
                         ref_He, 'H2O IWV', 'Correlation_H2O', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_H2O_HE/total_AIRM_HE, total_H2O_err_HE/total_AIRM_HE, indexes, 
                         ref_He, 'H2O IWV', 'Correlation_H2O', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_H2O/total_AIRM, total_H2O_err/total_AIRM, indexes, ref_He,
                             'H2O IWV', 'Correlation_H2O', False)

## Correlation between He triplet line and CO2 column density

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_CO2_HA/total_AIRM_HA, total_CO2_err_HA/total_AIRM_HA, indexes, 
                         ref_He, 'CO2 IWV', 'Correlation_CO2', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_CO2_HE/total_AIRM_HE, total_CO2_err_HE/total_AIRM_HE, indexes, 
                         ref_He, 'CO2 IWV', 'Correlation_CO2', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_CO2/total_AIRM, total_CO2_err/total_AIRM, indexes, ref_He,
                             'CO2 IWV', 'Correlation_CO2', False)

## Correlation between He triplet line and O2 column density

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_O2_HA/total_AIRM_HA, total_O2_err_HA/total_AIRM_HA, indexes,
                         ref_He, 'O2 IWV', 'Correlation_O2', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_O2_HE/total_AIRM_HE, total_O2_err_HE/total_AIRM_HE, indexes,
                         ref_He, 'O2 IWV', 'Correlation_O2', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_O2/total_AIRM, total_O2_err/total_AIRM, indexes, ref_He,
                             'O2 IWV', 'Correlation_O2', False)

## Correlation between He triplet line and airmass

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_AIRM_HA, np.zeros(total_AIRM_HA.shape), indexes, ref_He,
                         'Airmass', 'Correlation_airmass', False, True)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_AIRM_HE, np.zeros(total_AIRM_HE.shape), indexes, ref_He,
                         'Airmass', 'Correlation_airmass', False, True)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_AIRM, np.zeros(total_AIRM.shape), indexes, ref_He,
                             'Airmass', 'Correlation_airmass', False, True)

## Correlation between water line and He triplet

In [ ]:
reload(AF)

ref_T = 10833.3145 #From NIST database

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_mosaic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, theta_T_HA, err_theta_T_HA, indexes, ref_He, ref_T,
                         'He', 'Telluric', 'Correlation_He&T', False)
    
    #HE
    AF.Correlation_mosaic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, theta_T_HE, err_theta_T_HE, indexes, ref_He, ref_T,
                         'He', 'Telluric', 'Correlation_He&T', False)
else:
        AF.Correlation_mosaic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, theta_T, err_theta_T, indexes, ref_He, ref_T,
                             'He', 'Telluric', 'Correlation_He&T', False)

## Correlation between Si line and water line

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]

###################################################################################################################
###################################################################################################################
############################################ VOIGT ################################################################
###################################################################################################################
###################################################################################################################
print('VOIGT')
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_mosaic('HA', theta_Si_rassine_HA, err_theta_Si_rassine_HA,
                          theta_Si_Antaress_HA, err_theta_Si_Antaress_HA, theta_T_HA, err_theta_T_HA, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)
    
    #HE
    AF.Correlation_mosaic('HE', theta_Si_rassine_HE, err_theta_Si_rassine_HE,
                          theta_Si_Antaress_HE, err_theta_Si_Antaress_HE, theta_T_HE, err_theta_T_HE, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)
else:
    AF.Correlation_mosaic('', theta_Si_rassine, err_theta_Si_rassine,
                          theta_Si_Antaress, err_theta_Si_Antaress, theta_T, err_theta_T, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)
    
###################################################################################################################
###################################################################################################################
############################################ GAUSS ################################################################
###################################################################################################################
###################################################################################################################
print('GAUSSIAN')
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_mosaic('HA', thetaG_Si_rassine_HA, err_thetaG_Si_rassine_HA,
                          thetaG_Si_Antaress_HA, err_thetaG_Si_Antaress_HA, theta_T_HA, err_theta_T_HA, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)
    
    #HE
    AF.Correlation_mosaic('HE', thetaG_Si_rassine_HE, err_thetaG_Si_rassine_HE,
                          thetaG_Si_Antaress_HE, err_thetaG_Si_Antaress_HE, theta_T_HE, err_theta_T_HE, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)
else:
    AF.Correlation_mosaic('', thetaG_Si_rassine, err_thetaG_Si_rassine,
                          thetaG_Si_Antaress, err_thetaG_Si_Antaress, theta_T, err_theta_T, indexes, 
                          ref_Si, ref_T, 'Si', 'Telluric', 'Correlation_Si&T', False)


## Correlation between He triplet line and systematics

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, systematics_HA, np.zeros(systematics_HA.shape), indexes, ref_He,
                         'Modal Noise', 'Correlation_systematics', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, systematics_HE, np.zeros(systematics_HE.shape), indexes, ref_He,
                         'Modal Noise', 'Correlation_systematics', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, systematics, np.zeros(systematics.shape), indexes, ref_He,
                             'Modal Noise', 'Correlation_systematics', False)

## Correlation between Si line and Helium triplet line

In [ ]:
reload(AF)

ref_Si = AF.air2vac(10827.091) #From NIST database

indexes = [0, 1, 2, -1]

###################################################################################################################
###################################################################################################################
############################################ VOIGT ################################################################
###################################################################################################################
###################################################################################################################
print('NORMAL + VOIGT')
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_mosaic('HA', theta_rassine_HA, err_theta_rassine_HA,
                          theta_Antaress_HA, err_theta_Antaress_HA, theta_Si_Antaress_HA, err_theta_Si_Antaress_HA, indexes, 
                          ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)
    
    #HE
    AF.Correlation_mosaic('HE', theta_rassine_HE, err_theta_rassine_HE,
                          theta_Antaress_HE, err_theta_Antaress_HE, theta_Si_Antaress_HE, err_theta_Si_Antaress_HE, indexes, 
                          ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)
else:
    AF.Correlation_mosaic('', theta_rassine, err_theta_rassine,
                          theta_Antaress, err_theta_Antaress, theta_Si_Antaress, err_theta_Si_Antaress, indexes, 
                          ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)
    
###################################################################################################################
###################################################################################################################
############################################ GAUSS ################################################################
###################################################################################################################
###################################################################################################################
print('NORMAL + GAUSSIAN')
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_mosaic('HA', theta_rassine_HA, err_theta_rassine_HA,
                          theta_Antaress_HA, err_theta_Antaress_HA, thetaG_Si_Antaress_HA, err_thetaG_Si_Antaress_HA,
                          indexes, ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)
    
    #HE
    AF.Correlation_mosaic('HE', theta_rassine_HE, err_theta_rassine_HE,
                          theta_Antaress_HE, err_theta_Antaress_HE, thetaG_Si_Antaress_HE, err_thetaG_Si_Antaress_HE,
                          indexes, ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)
else:
    AF.Correlation_mosaic('', theta_rassine, err_theta_rassine,
                          theta_Antaress, err_theta_Antaress, thetaG_Si_Antaress, err_thetaG_Si_Antaress, indexes, 
                          ref_He, ref_Si, 'He', 'Si', 'Correlation_Si&He', False)


## Correlation between He triplet line and CCF Bisector Span

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_BIS_SPAN_HA, total_BIS_SPAN_err_HA, indexes, ref_He,
                         'CCF BIS SPAN (km/s)', 'Correlation_BIS_SPAN', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_BIS_SPAN_HE, total_BIS_SPAN_err_HE, indexes, ref_He,
                         'CCF BIS SPAN (km/s)', 'Correlation_BIS_SPAN', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_BIS_SPAN, total_BIS_SPAN_err, indexes, ref_He,
                             'CCF BIS SPAN (km/s)', 'Correlation_BIS_SPAN', False)

## Correlation between He triplet line and CCF FWHM

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_FWHM_HA, total_FWHM_err_HA, indexes, ref_He,
                         'CCF FWHM (km/s)', 'Correlation_FWHM', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_FWHM_HE, total_FWHM_err_HE, indexes, ref_He,
                         'CCF FWHM (km/s)', 'Correlation_FWHM', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_FWHM, total_FWHM_err, indexes, ref_He,
                             'CCF FWHM (km/s)', 'Correlation_FWHM', False)

## Correlation between He triplet line and CCF Contrast

In [ ]:
reload(AF)

indexes = [0, 1, 2, -1]
    
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    AF.Correlation_Basic('HA', theta_rassine_HA, err_theta_rassine_HA, theta_Antaress_HA, 
                        err_theta_Antaress_HA, total_CONTRAST_HA, total_CONTRAST_err_HA, indexes, ref_He,
                         'CCF CONTRAST', 'Correlation_CONTRAST', False)
    
    #HE
    AF.Correlation_Basic('HE', theta_rassine_HE, err_theta_rassine_HE, theta_Antaress_HE, 
                        err_theta_Antaress_HE, total_CONTRAST_HE, total_CONTRAST_err_HE, indexes, ref_He,
                         'CCF CONTRAST', 'Correlation_CONTRAST', False)
else:
        AF.Correlation_Basic('', theta_rassine, err_theta_rassine, theta_Antaress, 
                        err_theta_Antaress, total_CONTRAST, total_CONTRAST_err, indexes, ref_He,
                             'CCF CONTRAST', 'Correlation_CONTRAST', False)

## Random analyses of bad data

In [ ]:
'''reload(AF)

low = 10767
up = 10770
cap = 5
cap2 = 2

new_RV = tot_RV[tot_AIRM<cap2]
new_FWHM = tot_FWHM[tot_AIRM<cap2]
new_BIS_SPAN = tot_BIS_SPAN[tot_AIRM<cap2]
new_CONTRAST = tot_CONTRAST[tot_AIRM<cap2]
new_mode = tot_mode[tot_AIRM<cap2]

new_RV_err = tot_RV_err[tot_AIRM<cap2]
new_FWHM_err = tot_FWHM_err[tot_AIRM<cap2]
new_BIS_SPAN_err = tot_BIS_SPAN_err[tot_AIRM<cap2]
new_CONTRAST_err = tot_CONTRAST_err[tot_AIRM<cap2]


new_SNR = AF.SNR_calculator(low, up, tot_lamda[tot_AIRM<cap2], tot_spctr[tot_AIRM<cap2], tot_err[tot_AIRM<cap2])

#Correlation between SNR and RV
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_RV[new_SNR>cap], [], new_RV_err[new_SNR>cap], 'SNR', 'RV (km/s)', 'SNR-RV', day)

#Correlation between SNR and FWHM
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_FWHM[new_SNR>cap], [], new_FWHM_err[new_SNR>cap], 'SNR', 'FWHM (km/s)', 'SNR-FWHM', day)

#Correlation between SNR and BIS SPAN
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_BIS_SPAN[new_SNR>cap], [], new_BIS_SPAN_err[new_SNR>cap], 'SNR', 'Bisector Span (km/s)', 'SNR-Bisector Span', day)

#Correlation between SNR and CONTRAST
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_CONTRAST[new_SNR>cap], [], new_CONTRAST_err[new_SNR>cap], 'SNR', 'Contrast (%)', 'SNR-Contrast', day)

'''